# HiPPO Matrices
---

## Table of Contents
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
    * [Translated Legendre (LegT)](#translated-legendre-legt)
        * [LegT](#legt)
        * [LMU](#lmu)
    * [Translated Laguerre (LagT)](#translated-laguerre-lagt)
    * [Scaled Legendre (LegS)](#scaled-legendre-legs)
    * [Fourier Basis](#fourier-basis)
        * [Fourier Recurrent Unit (FRU)](#fourier-recurrent-unit-fru)
        * [Truncated Fourier (FouT)](#truncated-fourier-fout)
        * [Fourier With Decay (FourD)](#fourier-with-decay-fourd)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
    * [Testing Forward Euler on GBT matrices](#testing-forward-euler-transform-for-lti-and-lsi)
    * [Testing Backward Euler on GBT matrices](#testing-backward-euler-transform-for-lti-and-lsi-on-legs-matrices)
    * [Testing Bidirectional on GBT matrices](#testing-lti-and-lsi-operators-with-bidirectional-transform)
    * [Testing ZOH on GBT matrices](#testing-zoh-transform-for-lti-and-lsi-on-legs-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
    * [Testing Forward Euler on HiPPO Operators](#testing-lti-and-lsi-operators-with-forward-euler-transform)
    * [Testing Backward Euler on HiPPO Operators](#testing-lti-and-lsi-operators-with-backward-euler-transform)
    * [Testing Bidirectional on HiPPO Operators](#testing-lti-and-lsi-operators-with-bidirectional-transform)
    * [Testing ZOH on HiPPO Operators](#testing-lti-and-lsi-operators-with-zoh-transform)
---


## Load Packages

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("../../../"))
print(f"module_path: {module_path}")
if module_path not in sys.path:
    print(f"Adding {module_path} to sys.path")
    sys.path.append(module_path)

module_path: /home/beegass/Documents/Coding/s4mer


In [2]:
os.environ["TF_FORCE_UNIFIED_MEMORY"] = "1"

In [3]:
## import packages
import jax
import jax.numpy as jnp
from src.models.hippo.hr_hippo import HRHiPPO_LSI, HRHiPPO_LTI
from src.models.hippo.hippo import HiPPOLSI, HiPPOLTI
from src.data.process import whitesignal
import einops
import numpy as np
import torch

/home/beegass/.cache/pypoetry/virtualenvs/s4mer-pkg-jZnBSgjq-py3.8/lib/python3.8/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/beegass/.cache/pypoetry/virtualenvs/s4mer-pkg-jZnBSgjq-py3.8/lib/python3.8/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [4]:
print(jax.devices())
print(f"The Device: {jax.lib.xla_bridge.get_backend().platform}")

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]
The Device: gpu


In [5]:
print(f"MPS enabled: {torch.backends.mps.is_available()}")

MPS enabled: False


In [6]:
torch.set_printoptions(linewidth=150)
np.set_printoptions(linewidth=150)
jnp.set_printoptions(linewidth=150)

In [7]:
seed = 1701
key = jax.random.PRNGKey(seed)

In [12]:
num_copies = 10
subkeys = jax.random.split(key, num=num_copies)
key = subkeys[0]

## Test HiPPO Reconstruction

In [13]:
def test_hippo_reconstruction(
    hippo, gu_hippo, random_input, key, s_or_t="lti", print_all=False
):
    x_tensor = torch.tensor(random_input, dtype=torch.float32)
    x_jnp = jnp.asarray(x_tensor, dtype=jnp.float32)  # convert torch array to jax array

    # My Implementation
    if print_all:
        print(
            f"------------------------------------------------------------------------------------------"
        )
        print(
            f"----------------------------My {s_or_t} Implementation Outputs----------------------------"
        )
        print(
            f"------------------------------------------------------------------------------------------"
        )
    x_jnp = einops.rearrange(x_jnp, "batch seq_len -> batch seq_len 1")
    params = hippo.init(key, f=x_jnp)
    hippo = hippo.bind(params)
    c = hippo.__call__(f=x_jnp)
    y = hippo.reconstruct(c)

    # Gu's HiPPO LegS
    if print_all:
        print(
            f"------------------------------------------------------------------------------------------"
        )
        print(
            f"---------------------------Gu's {s_or_t} Implementation Outputs---------------------------"
        )
        print(
            f"------------------------------------------------------------------------------------------"
        )
    x_tensor = torch.moveaxis(x_tensor, 0, 1)
    GU_c_s, GU_c_k = gu_hippo(x_tensor, fast=False)

    gu_y = None
    if s_or_t == "lsi":
        GU_c_s = einops.rearrange(GU_c_s, "seq_len batch N -> batch seq_len 1 N")
        gu_y = gu_hippo.reconstruct(GU_c_s)
    elif s_or_t == "lti":
        GU_c_k = einops.rearrange(GU_c_k, "batch N -> batch 1 N")
        gu_y = gu_hippo.reconstruct(GU_c_k)
    else:
        raise ValueError(
            f"s_or_t must be either 'lsi' or 'lti'. s_or_t is currently set to: {s_or_t}"
        )

    if s_or_t == "lsi":
        gu_c = jnp.asarray(
            GU_c_s, dtype=jnp.float32
        )  # convert torch array to jax array
        # (2, 3000, 3000, 1)
        y = einops.rearrange(
            y, "batch seq_len seq_len2 input_len -> batch seq_len input_len seq_len2"
        )
        gu_y = einops.rearrange(
            gu_y, "seq_len batch seq_len2 input_len -> batch seq_len input_len seq_len2"
        )

    elif s_or_t == "lti":
        gu_c = jnp.asarray(
            GU_c_k, dtype=jnp.float32
        )  # convert torch array to jax array
        # (1, 64, 1)
        y = einops.rearrange(
            y, "batch seq_len -> batch 1 seq_len"
        )  # transposing
        gu_y = einops.rearrange(
            gu_y, "batch seq_len -> batch 1 seq_len"
        )  # transposing

    gu_y = jnp.asarray(gu_y, dtype=jnp.float32)  # convert torch array to jax array

    co_flag = True
    y_flag = True
    for i in range(c.shape[0]):
        for j in range(c.shape[1]):

            co_check = None
            y_check = None
            if s_or_t == "lsi":
                co_check = jnp.allclose(
                    c[i, j, :, :], gu_c[i, j, :, :], rtol=1e-03, atol=1e-03
                )
                y_check = jnp.allclose(
                    y[i, j, :, :], gu_y[i, j, :, :], rtol=1e-03, atol=1e-03
                )

                if print_all:
                    print(f"y[{i},{j},:,:]:\n{y[i,j,:,:]}")
                    print(f"gu_y[{i},{j},:,:]:\n{gu_y[i,j,:,:]}")
                    print(f"reconstruction check: {y_check}\n")

            elif s_or_t == "lti":
                co_check = jnp.allclose(
                    c[i, j, :], gu_c[i, :, :], rtol=1e-03, atol=1e-03
                )
                y_check = jnp.allclose(
                    y[i, j, :], gu_y[i, :, :], rtol=1e-03, atol=1e-03
                )

                if print_all:
                    print(f"y[{i},{j},:]:\n{y[i,:,:]}")
                    print(f"gu_y[{i},{j},:]:\n{gu_y[i,:,:]}")
                    print(f"reconstruction check: {y_check}\n")

            if co_check == False:
                co_flag = False

            if y_check == False:
                y_flag = False

    if not print_all:
        print(f"\n------------------------------------------------------------")
        print(f"---------- The Coefficients Test Passed: {co_flag} ----------")
        print(f"-------------------------------------------------------------\n\n")
        print(f"\n-------------------------------------------------------------")
        print(f"---------- The Reconstruction Test Passed: {y_flag} ----------")
        print(f"--------------------------------------------------------------\n")

    if print_all:
        print(f"y:\n {y}")
        print(f"y shape:\n {y.shape}")

        print(f"gu_y:\n {gu_y}")
        print(f"gu_y shape:\n {gu_y.shape}")

In [14]:
def test_reconstruction(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=False
):
    # N = 256
    # L = 128

    T = 3
    freq = 3
    step = 1e-3
    L = int(T / step)

    batch_size = 2
    data_size = L
    input_size = 1

    N = 64

    u = whitesignal(subkeys[4], T, step, freq, batch_shape=(batch_size,))

    x_np = np.asarray(u)

    # ----------------------------------------------------------------------------------
    # ------------------------------ Instantiate Gu's HiPPOs ---------------------------
    # ----------------------------------------------------------------------------------

    print(f"Creating Gu's HiPPO-{the_measure} LTI model with {alpha} transform")
    gu_hippo_lti = HRHiPPO_LTI(
        N=N,
        method=the_measure,
        dt=step,
        T=T,
        discretization=discretization,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        c=0.0,
    )  # The Gu's

    if the_measure == "legs":
        print(f"Creating Gu's HiPPO-{the_measure} LSI model with {alpha} transform")
        gu_hippo_lsi = HRHiPPO_LSI(
            N=N,
            method=the_measure,
            max_length=L,
            discretization=discretization,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
        )  # The Gu's

    # ----------------------------------------------------------------------------------
    # ------------------------------ Instantiate My HiPPOs -----------------------------
    # ----------------------------------------------------------------------------------
    print(f"Creating HiPPO-{the_measure} LTI model with {alpha} transform")
    hippo_lti = HiPPOLTI(
        N=N,
        step_size=step,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        GBT_alpha=alpha,
        measure=the_measure,
        basis_size=T,
        dtype=jnp.float32,
        unroll=False,
    )

    if the_measure == "legs":
        print(f"Creating HiPPO-{the_measure} LSI model with {alpha} transform")
        hippo_lsi = HiPPOLSI(
            N=N,
            max_length=L,
            step_size=step,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
            GBT_alpha=alpha,
            measure=the_measure,
            dtype=jnp.float32,
            unroll=True,
        )

    # ----------------------------------------------------------------------------------
    # ------------------------------ Test HiPPO Reconstruction -------------------------
    # ----------------------------------------------------------------------------------

    print(f"Testing Reconstruction for {alpha} LTI HiPPO-{the_measure}")

    test_hippo_reconstruction(
        hippo=hippo_lti,
        gu_hippo=gu_hippo_lti,
        random_input=x_np,
        key=subkeys[5],
        s_or_t="lti",
        print_all=print_all,
    )

    if the_measure == "legs":
        print(f"\n\nTesting Reconstruction for {alpha} LSI HiPPO-{the_measure}")

        test_hippo_reconstruction(
            hippo=hippo_lsi,
            gu_hippo=gu_hippo_lsi,
            random_input=x_np,
            key=subkeys[6],
            s_or_t="lsi",
            print_all=print_all,
        )

    print(f"end of test for HiPPO-{the_measure} model")

## Navigation To Table Of Contents
---
* [Table Of Contents](#table-of-contents)
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
---

In [15]:
print_all = False

### Testing (LTI and LSI) Operators With Forward Euler Transform

#### LegS

In [16]:
test_reconstruction(
    the_measure="legs", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 0.0 transform
Creating Gu's HiPPO-legs LSI model with 0.0 transform
Creating HiPPO-legs LTI model with 0.0 transform
Creating HiPPO-legs LSI model with 0.0 transform
Testing Reconstruction for 0.0 LTI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------



Testing Reconstruction for 0.0 LSI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
------------------------

#### LegT

In [13]:
test_reconstruction(
    the_measure="legt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 0.0 transform
Creating HiPPO-legt LTI model with 0.0 transform
Testing Reconstruction for 0.0 LTI HiPPO-legt


/home/beegass/.cache/pypoetry/virtualenvs/s4mer-pkg-jZnBSgjq-py3.8/lib/python3.8/site-packages/jax/_src/numpy/lax_numpy.py:5117: RuntimeWarning: overflow encountered in cast
  return binary_op(*args)



------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-legt model


#### LMU

In [14]:
test_reconstruction(
    the_measure="lmu", lambda_n=2.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 0.0 transform
Creating HiPPO-lmu LTI model with 0.0 transform
Testing Reconstruction for 0.0 LTI HiPPO-lmu

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-lmu model


#### LagT

In [15]:
test_reconstruction(
    the_measure="lagt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 0.0 transform
Creating HiPPO-lagt LTI model with 0.0 transform
Testing Reconstruction for 0.0 LTI HiPPO-lagt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-lagt model


#### FouT

In [16]:
test_reconstruction(
    the_measure="fout", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 0.0 transform
Creating HiPPO-fout LTI model with 0.0 transform
Testing Reconstruction for 0.0 LTI HiPPO-fout

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-fout model


### Testing (LTI and LSI) Operators With Backward Euler Transform

#### LegS

In [17]:
test_reconstruction(
    the_measure="legs", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 1.0 transform
Creating Gu's HiPPO-legs LSI model with 1.0 transform
Creating HiPPO-legs LTI model with 1.0 transform
Creating HiPPO-legs LSI model with 1.0 transform
Testing Reconstruction for 1.0 LTI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------



Testing Reconstruction for 1.0 LSI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
-------------------------

#### LegT

In [18]:
test_reconstruction(
    the_measure="legt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 1.0 transform
Creating HiPPO-legt LTI model with 1.0 transform
Testing Reconstruction for 1.0 LTI HiPPO-legt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-legt model


#### LMU

In [19]:
test_reconstruction(
    the_measure="lmu", lambda_n=2.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 1.0 transform
Creating HiPPO-lmu LTI model with 1.0 transform
Testing Reconstruction for 1.0 LTI HiPPO-lmu

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-lmu model


#### LagT

In [20]:
test_reconstruction(
    the_measure="lagt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 1.0 transform
Creating HiPPO-lagt LTI model with 1.0 transform
Testing Reconstruction for 1.0 LTI HiPPO-lagt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-lagt model


#### FouT

In [21]:
test_reconstruction(
    the_measure="fout", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 1.0 transform
Creating HiPPO-fout LTI model with 1.0 transform
Testing Reconstruction for 1.0 LTI HiPPO-fout

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-fout model


### Testing (LTI and LSI) Operators With Bidirectional Transform

#### LegS

In [22]:
test_reconstruction(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 0.5 transform
Creating Gu's HiPPO-legs LSI model with 0.5 transform
Creating HiPPO-legs LTI model with 0.5 transform
Creating HiPPO-legs LSI model with 0.5 transform
Testing Reconstruction for 0.5 LTI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------



Testing Reconstruction for 0.5 LSI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
-------------------------

#### LegT

In [23]:
test_reconstruction(
    the_measure="legt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 0.5 transform
Creating HiPPO-legt LTI model with 0.5 transform
Testing Reconstruction for 0.5 LTI HiPPO-legt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-legt model


#### LMU

In [24]:
test_reconstruction(
    the_measure="lmu", lambda_n=2.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 0.5 transform
Creating HiPPO-lmu LTI model with 0.5 transform
Testing Reconstruction for 0.5 LTI HiPPO-lmu

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-lmu model


#### LagT

In [25]:
test_reconstruction(
    the_measure="lagt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 0.5 transform
Creating HiPPO-lagt LTI model with 0.5 transform
Testing Reconstruction for 0.5 LTI HiPPO-lagt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-lagt model


#### FouT

In [26]:
test_reconstruction(
    the_measure="fout", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 0.5 transform
Creating HiPPO-fout LTI model with 0.5 transform
Testing Reconstruction for 0.5 LTI HiPPO-fout

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-fout model


### Testing (LTI and LSI) Operators With ZOH Transform

#### LegS

In [27]:
test_reconstruction(
    the_measure="legs",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-legs LTI model with 2.0 transform
Creating Gu's HiPPO-legs LSI model with 2.0 transform
Creating HiPPO-legs LTI model with 2.0 transform
Creating HiPPO-legs LSI model with 2.0 transform
Testing Reconstruction for 2.0 LTI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------



Testing Reconstruction for 2.0 LSI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
-------------------------

#### LegT

In [28]:
test_reconstruction(
    the_measure="legt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-legt LTI model with 2.0 transform
Creating HiPPO-legt LTI model with 2.0 transform
Testing Reconstruction for 2.0 LTI HiPPO-legt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-legt model


#### LMU

In [29]:
test_reconstruction(
    the_measure="lmu",
    lambda_n=2.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-lmu LTI model with 2.0 transform
Creating HiPPO-lmu LTI model with 2.0 transform
Testing Reconstruction for 2.0 LTI HiPPO-lmu

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-lmu model


#### LagT

In [30]:
test_reconstruction(
    the_measure="lagt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-lagt LTI model with 2.0 transform
Creating HiPPO-lagt LTI model with 2.0 transform
Testing Reconstruction for 2.0 LTI HiPPO-lagt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-lagt model


#### FouT

In [31]:
test_reconstruction(
    the_measure="fout",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-fout LTI model with 2.0 transform
Creating HiPPO-fout LTI model with 2.0 transform
Testing Reconstruction for 2.0 LTI HiPPO-fout

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-fout model
